<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2_and_i2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>






























## **Diffusers ライブラリを用いて、画像を生成するスクリプト。**
参考 ：
[日本語](https://blog.shikoan.com/controlnet_lora/#ControlNet%EF%BC%88%E3%83%9D%E3%83%BC%E3%82%BA%EF%BC%89%E3%82%92%E4%BD%BF%E3%81%86)  ,
[英語](https://huggingface.co/blog/controlnet) ,
[Github - AA-google-colab-kohya](https://github.com/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb)


In [ ]:
#@title Driveに接続, ライブラリの追加 { display-mode: "form" }

# Driveに接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ライブラリの追加
# 最新版（0.18.0）
!pip install git+https://github.com/huggingface/diffusers.git@v0.18.0 >/dev/null 2>&1
!pip install --upgrade transformers accelerate scipy ftfy xformers safetensors txt2img >/dev/null 2>&1 # diffusers==0.17.1
!pip install k-diffusion OmegaConf opencv-contrib-python controlnet_aux >/dev/null 2>&1


In [ ]:
#@title 自動切断されないようにするコード { display-mode: "form" }
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
#@title ライブラリをインポート + S2D2 / Embeddingsの準備 { display-mode: "form" }
%cd /content
!git clone https://github.com/keisuke-okb/S2D2 &> /dev/null

%cd ./S2D2
!git pull
!touch __init__.py
!pip install -r requirements.txt >/dev/null 2>&1


# 各ライブラリをインポート
import s2d2

# diffusers ライブラリをインポートする
import diffusers
from diffusers import (StableDiffusionPipeline,StableDiffusionImg2ImgPipeline)

# PyTorchという深層学習フレームワークをインポートする
import torch


##################################################


import os
import random
import datetime
import torch
from PIL import Image

import diffusers
from diffusers import (StableDiffusionPipeline,
                      StableDiffusionImg2ImgPipeline)
from diffusers.utils import numpy_to_pil
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

import torch
import datetime
from PIL import Image
import numpy as np

SCHEDULERS = {
    "unipc": diffusers.schedulers.UniPCMultistepScheduler,
    "euler_a": diffusers.schedulers.EulerAncestralDiscreteScheduler,
    "euler": diffusers.schedulers.EulerDiscreteScheduler,
    "ddim": diffusers.schedulers.DDIMScheduler,
    "ddpm": diffusers.schedulers.DDPMScheduler,
    "deis": diffusers.schedulers.DEISMultistepScheduler,
    "dpm2": diffusers.schedulers.KDPM2DiscreteScheduler,
    "dpm2-a": diffusers.schedulers.KDPM2AncestralDiscreteScheduler,
    "dpm++_2s": diffusers.schedulers.DPMSolverSinglestepScheduler,
    "dpm++_2m": diffusers.schedulers.DPMSolverMultistepScheduler,
    "dpm++_2m_karras": diffusers.schedulers.DPMSolverMultistepScheduler,
    "dpm++_sde": diffusers.schedulers.DPMSolverSDEScheduler,
    "dpm++_sde_karras": diffusers.schedulers.DPMSolverSDEScheduler,
    "heun": diffusers.schedulers.HeunDiscreteScheduler,
    "heun_karras": diffusers.schedulers.HeunDiscreteScheduler,
    "lms": diffusers.schedulers.LMSDiscreteScheduler,
    "lms_karras": diffusers.schedulers.LMSDiscreteScheduler,
    "pndm": diffusers.schedulers.PNDMScheduler,
}

def calc_pix_8(x):
    x = int(x)
    return x - x % 8

##################################################

class StableDiffusionImageGeneratorMod(s2d2.StableDiffusionImageGenerator):
  def __init__(self, sd_safetensor_path: str, device: str="cuda", dtype: torch.dtype=torch.float16, vae=None):

    self.device = torch.device(device)
    self.vae = vae

    self.pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      vae=vae,
      custom_pipeline="lpw_stable_diffusion",
    ).to(device)

    self.pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      vae=vae,
      custom_pipeline="lpw_stable_diffusion",
    ).to(device)

    self.pipe.enable_xformers_memory_efficient_attention()
    self.pipe_i2i.enable_xformers_memory_efficient_attention()

    self.pipe.safety_checker = None
    self.pipe_i2i.safety_checker = None
    return

  def load_embeddings(self, safetensors_path: str, fileName: str, token: str):
    if token != "":
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
    else:
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName)
    return


###########################################


  def decode_latents_to_PIL_image(self, latents, decode_factor=0.18215):
      with torch.no_grad():
          latents = 1 / decode_factor * latents
          image = self.pipe.vae.decode(latents).sample
          image = (image / 2 + 0.5).clamp(0, 1)
          image = image.cpu().permute(0, 2, 3, 1).float().numpy()
          image = numpy_to_pil(image)
          image = StableDiffusionPipelineOutput(images=image, nsfw_content_detected=None).images[0]
          return image


  def diffusion_from_noise(
          self,
          prompt,
          negative_prompt,
          image,
          scheduler_name="dpm++_2m_karras",
          num_inference_steps=20,
          guidance_scale=9.5,
          width=512,
          height=512,
          output_type="pil",
          decode_factor=0.18215,
          seed=1234,
          max_embeddings_multiples=None,
          save_path=None
          ):

      self.pipe.scheduler = SCHEDULERS[scheduler_name].from_config(self.pipe.scheduler.config)
      self.pipe.scheduler.set_timesteps(num_inference_steps, self.device)
      seed = random.randint(1, 1000000000) if seed == -1 else seed

      with torch.no_grad():
          latents = self.pipe(
              prompt=prompt,
              negative_prompt=negative_prompt,
              image=image,
              num_inference_steps=num_inference_steps,
              generator=torch.manual_seed(seed),
              guidance_scale=guidance_scale,
              max_embeddings_multiples=max_embeddings_multiples,
              width=width,
              height=height,
              output_type="latent"
          ).images # 1x4x(W/8)x(H/8)

          if save_path is not None:
              pil_image = self.decode_latents_to_PIL_image(latents, decode_factor)
              os.makedirs(os.path.dirname(save_path), exist_ok=True)
              pil_image.save(save_path, quality=95)

          if output_type == "latent":
              return latents
          elif output_type == "pil":
              return self.decode_latents_to_PIL_image(latents, decode_factor)
          else:
              raise NotImplementedError()

  def diffusion_from_image(
          self,
          prompt,
          negative_prompt,
          image,
          scheduler_name="dpm++_2m_karras",
          num_inference_steps=20,
          denoising_strength=0.58,
          guidance_scale=10,
          output_type="pil",
          decode_factor=0.18215,
          seed=1234,
          max_embeddings_multiples=None,
          save_path=None
          ):

      self.pipe_i2i.scheduler = SCHEDULERS[scheduler_name].from_config(self.pipe_i2i.scheduler.config)
      self.pipe_i2i.scheduler.set_timesteps(num_inference_steps, self.device)
      seed = random.randint(1, 1000000000) if seed == -1 else seed

      with torch.no_grad():
          latents = self.pipe_i2i(
              prompt=prompt,
              negative_prompt=negative_prompt,
              image=image,
              num_inference_steps=num_inference_steps,
              strength=denoising_strength,
              generator=torch.manual_seed(seed),
              max_embeddings_multiples=max_embeddings_multiples,
              guidance_scale=guidance_scale,
              output_type="latent"
          ).images # 1x4x(W/8)x(H/8)

          if save_path is not None:
              pil_image = self.decode_latents_to_PIL_image(latents, decode_factor)
              os.makedirs(os.path.dirname(save_path), exist_ok=True)
              pil_image.save(save_path, quality=95)

          if output_type == "latent":
              return latents
          elif output_type == "pil":
              return self.decode_latents_to_PIL_image(latents, decode_factor)
          else:
              raise NotImplementedError()


  def diffusion_enhance(
          self,
          prompt,
          negative_prompt,
          image,
          scheduler_name="dpm++_2m_karras",
          num_inference_steps=20,
          num_inference_steps_enhance=20,
          guidance_scale=10,
          width=512,
          height=512,
          seed=1234,
          max_embeddings_multiples=None,
          upscale_target="latent", # "latent" or "pil"
          interpolate_mode="nearest",
          antialias = True,
          upscale_by=1.8,
          enhance_steps=2, # 2=Hires.fix
          denoising_strength=0.58,
          output_type="pil",
          decode_factor=0.15,
          decode_factor_final=0.18215,
          save_dir="output"
          ):

      with torch.no_grad():
          w_init = calc_pix_8(width)
          h_init = calc_pix_8(height)
          w_final = calc_pix_8(w_init * upscale_by)
          h_final = calc_pix_8(h_init * upscale_by)
          resolution_pairs = [(calc_pix_8(x), calc_pix_8(y)) for x, y
                  in zip(np.linspace(w_init, w_final, enhance_steps),
                          np.linspace(h_init, h_final, enhance_steps))
                  ]
          image = None
          now_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

          if enhance_steps == 1: # Single generation
              image = self.diffusion_from_noise(
                      prompt,
                      negative_prompt,
                      image,
                      scheduler_name=scheduler_name,
                      num_inference_steps=num_inference_steps,
                      guidance_scale=guidance_scale,
                      width=w_final,
                      height=h_final,
                      output_type=output_type,
                      decode_factor=decode_factor_final,
                      seed=seed,
                      max_embeddings_multiples=max_embeddings_multiples,
                      save_path=os.path.join(save_dir, f"{now_str}.jpg")
                  )
              return image


          for i, (w, h) in enumerate(resolution_pairs):

              if image is None: # Step 1: Generate low-quality image
                  image = self.diffusion_from_noise(
                      prompt,
                      negative_prompt,
                      image,
                      scheduler_name=scheduler_name,
                      num_inference_steps=num_inference_steps,
                      guidance_scale=guidance_scale,
                      width=w,
                      height=h,
                      output_type=upscale_target,
                      decode_factor=decode_factor,
                      seed=seed,
                      max_embeddings_multiples=max_embeddings_multiples,
                      save_path=os.path.join(save_dir, f"{now_str}_{i}.jpg")
                  )
                  continue

              # Step 2: Interpolate latent or image -> PIL image
              if upscale_target == "latent":
                  image = torch.nn.functional.interpolate(
                          image,
                          (h // 8, w // 8),
                          mode=interpolate_mode,
                          antialias=True if antialias and interpolate_mode != "nearest" else False,
                      )
                  image = self.decode_latents_to_PIL_image(image, decode_factor)
              else:
                  image = image.resize((w, h), Image.Resampling.LANCZOS)

              # Step 3: Generate image (i2i)
              if i < len(resolution_pairs) - 1:
                  image = self.diffusion_from_image(
                      prompt,
                      negative_prompt,
                      image,
                      scheduler_name=scheduler_name,
                      num_inference_steps=int(num_inference_steps_enhance / denoising_strength) + 1,
                      denoising_strength=denoising_strength,
                      guidance_scale=guidance_scale,
                      output_type=upscale_target,
                      decode_factor=decode_factor,
                      seed=seed,
                      max_embeddings_multiples=max_embeddings_multiples,
                      save_path=os.path.join(save_dir, f"{now_str}_{i}.jpg")
                  )

              else: # Final enhance
                  image = self.diffusion_from_image(
                      prompt,
                      negative_prompt,
                      image,
                      scheduler_name=scheduler_name,
                      num_inference_steps=int(num_inference_steps_enhance / denoising_strength) + 1,
                      denoising_strength=denoising_strength,
                      guidance_scale=guidance_scale,
                      output_type=output_type,
                      decode_factor=decode_factor_final,
                      seed=seed,
                      max_embeddings_multiples=max_embeddings_multiples,
                      save_path=os.path.join(save_dir, f"{now_str}_{i}.jpg")
                  )
                  return image

  %cd /content/


In [ ]:
# @title Model、LoRA を StableDiffusionImageGeneratorMod に設定 { display-mode: "form" }


#モデル
# @markdown ***
# @markdown 👚 Model
model_dir = "/content/drive/MyDrive/StableDiffusion/Model/"
model_name = "beautifulRealistic_v60" #@param ["kanpiromix_v20","chilled_remix_v2", "emilianJR/XXMix_9realistic", "beautifulRealistic_v60", "sinkinai/Beautiful-Realistic-Asians-v5", "emilianJR/majicMIX_realistic_v6", "runwayml/stable-diffusion-v1-5", "stablediffusionapi/anything-v5"] {allow-input: true}
model_path = os.path.join(model_dir, model_name)
# 「/」が含まれていたら、〇〇を実行する
if "/" in model_name:
  model_path = model_name

# VAE を読み込む
vae_path = "stabilityai/sd-vae-ft-mse" #@param ["stabilityai/sd-vae-ft-mse","stabilityai/sd-vae-ft-ema", "rossiyareich/anything-v4.0-vae"] {allow-input: true}
vae=AutoencoderKL.from_pretrained(pretrained_model_name_or_path=vae_path, torch_dtype=torch.float16)

generator = StableDiffusionImageGeneratorMod(
  model_path,
  controlnet=controlnet,
  vae=vae,
)

# @markdown ***
# @markdown 🙎 LoRA  →  ファイル名に  .safetensors  をつけること。
lora_base_dir = "/content/drive/MyDrive/StableDiffusion/Lora/"

isUse_lora_1 = True #@param {type:"boolean"}
if isUse_lora_1:
  lora_1="flat2.safetensors" #@param ["flat2", "add_detail", "koreanDollLikeness_v20", "japaneseDollLikeness_v15", "handsome_male-02", "trueBuruma_v26red", "Bukkake", "fixStandingDoggyStyle-000017", "concept-staged-deepthroat-v3", "rum_and_koya", "koya_v6"] {allow-input: true}
  lora_alpha_1 = -1 # @param {type:"slider", min:-1, max:2, step:0.1}
  lora_path_1 = os.path.join(lora_base_dir, lora_1)
  generator.load_lora(lora_path_1, alpha=lora_alpha_1)

isUse_lora_2= True #@param {type:"boolean"}
if isUse_lora_2:
  lora_2="haruchan_sd_no_req_add_vae_trans_bakground_AIorigin-000030.safetensors" #@param ["flat2", "add_detail", "koreanDollLikeness_v20", "japaneseDollLikeness_v15", "handsome_male-02", "trueBuruma_v26red", "Bukkake", "fixStandingDoggyStyle-000017", "concept-staged-deepthroat-v3", "rum_and_koya", "koya_v6"] {allow-input: true}
  lora_alpha_2 = 0.5 # @param {type:"slider", min:-1, max:2, step:0.1}
  lora_path_2 = os.path.join(lora_base_dir, lora_2)
  generator.load_lora(lora_path_2, alpha=lora_alpha_2)

isUse_lora_3= False #@param {type:"boolean"}
if isUse_lora_3:
  lora_3="WeddingDressEXv0.4.safetensors" #@param ["flat2", "add_detail", "koreanDollLikeness_v20", "japaneseDollLikeness_v15", "handsome_male-02", "trueBuruma_v26red", "Bukkake", "fixStandingDoggyStyle-000017", "concept-staged-deepthroat-v3", "rum_and_koya", "koya_v6"] {allow-input: true}
  lora_alpha_3 = 0.4 # @param {type:"slider", min:-1, max:2, step:0.1}
  lora_path_3 = os.path.join(lora_base_dir, lora_3)
  generator.load_lora(lora_path_3, alpha=lora_alpha_3)

isUse_lora_4= False #@param {type:"boolean"}
if isUse_lora_4:
  lora_4="dreamflower.safetensors" #@param ["flat2", "add_detail", "koreanDollLikeness_v20", "japaneseDollLikeness_v15", "handsome_male-02", "trueBuruma_v26red", "Bukkake", "fixStandingDoggyStyle-000017", "concept-staged-deepthroat-v3", "rum_and_koya", "koya_v6"] {allow-input: true}
  lora_alpha_4 = 0.2 # @param {type:"slider", min:-1, max:2, step:0.1}
  lora_path_4 = os.path.join(lora_base_dir, lora_4)
  generator.load_lora(lora_path_4, alpha=lora_alpha_4)

isUse_lora_5= True #@param {type:"boolean"}
if isUse_lora_5:
  lora_5="japaneseDollLikeness_v15.safetensors" #@param ["flat2", "add_detail", "koreanDollLikeness_v20", "japaneseDollLikeness_v15", "handsome_male-02", "trueBuruma_v26red", "Bukkake", "fixStandingDoggyStyle-000017", "concept-staged-deepthroat-v3", "rum_and_koya", "koya_v6"] {allow-input: true}
  lora_alpha_5 = 0.3 # @param {type:"slider", min:-1, max:2, step:0.1}
  lora_path_5 = os.path.join(lora_base_dir, lora_5)
  generator.load_lora(lora_path_5, alpha=lora_alpha_5)

# @markdown ***
#@markdown 👾 Embeddings
embeddings_base_dir = "/content/drive/MyDrive/StableDiffusion/embeddings/"

isUse_embedding_1 = False #@param {type:"boolean"}
if isUse_embedding_1:
  embedding_1 = "EasyNegative.safetensors" #@param {type:"string"}
  trigger_word_1 = "EasyNegative" #@param {type:"string"}
  safetensors_path_1 = os.path.join(embeddings_base_dir, embedding_1)
  generator.load_embeddings(safetensors_path_1, embedding_1, trigger_word_1)

isUse_embedding_2 = True #@param {type:"boolean"}
if isUse_embedding_2:
  embedding_2 = "negative_hand-neg.pt" #@param {type:"string"}
  trigger_word_2 = "negative_hand-neg" #@param {type:"string"}
  safetensors_path_2 = os.path.join(embeddings_base_dir, embedding_2)
  generator.load_embeddings(safetensors_path_2, embedding_2, trigger_word_2)

isUse_embedding_3 = False #@param {type:"boolean"}
if isUse_embedding_3:
  embedding_3 = "ulzzang-6500-v1.1.pt" #@param {type:"string"}
  trigger_word_3 = "ulzzang-6500-v1.1" #@param {type:"string"}
  safetensors_path_3 = os.path.join(embeddings_base_dir, embedding_3)
  generator.load_embeddings(safetensors_path_3, embedding_3, trigger_word_3)


isUse_embedding_4 = True #@param {type:"boolean"}
if isUse_embedding_4:
  embedding_4 = "ng_deepnegative_v1_75t.pt" #@param {type:"string"}
  trigger_word_4 = "ng_deepnegative_v1_75t" #@param {type:"string"}
  safetensors_path_4 = os.path.join(embeddings_base_dir, embedding_4)
  generator.load_embeddings(safetensors_path_4, embedding_4, trigger_word_4)


isUse_embedding_5 = False #@param {type:"boolean"}
if isUse_embedding_5:
  embedding_5 = "GS-DeFeminize-neg.pt" #@param {type:"string"}
  trigger_word_5 = "GS-Masculine, GS-DeFeminize-Neg" #@param {type:"string"}
  safetensors_path_5 = os.path.join(embeddings_base_dir, embedding_5)
  generator.load_embeddings(safetensors_path_5, embedding_4, trigger_word_5)


# @markdown ***


In [ ]:
# @title 画像を生成  { display-mode: "form" }
from PIL.PngImagePlugin import PngInfo

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

# 現在の日本時間を取得
jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#txt2img出力画像の保存先
#@markdown ***出力画像を保存するフォルダ***
save_dir = "/content/drive/MyDrive/StableDiffusion/txt2img_output/"
save_folder = "wedding" #@param {type: "string"}
save_path = os.path.join(save_dir, save_folder)
os.makedirs(save_path, exist_ok=True)


#@@markdown ポジティブプロンプト
prompt = "((masterpiece)), ((best quality)), (photo realistic), ((haruchan)), small Breast, holding flower, ((full body, in superb view)), (wedding dress, veil:1.2), Earrings, SunLight" #@param {type:"string"}
#@@markdown ネガティブプロンプト
negative = "((worst picture quality, low quality, normal quality)), ((negative_hand)), ((negative_hand-neg)), ((ng_deepnegative_v1_75t)),  painting, sketch, text, error, jpeg image, logo, signature, watermark, Username, (extra fingers, deformed hands, polydactyl:1.5)" #@param {type:"string"}
#@markdown ***出力枚数***
batch_count = 5 # @param {type:"slider", min:1, max:100, step:1}
#@markdown ***ステップ数***
steps = 20 # @param {type:"slider", min:5, max:100, step:5}
#@@markdown 画像サイズ
img_width =   512  # @param {type:"slider", min:256, max:1920, step:64}
img_height =  768  # @param {type:"slider", min:256, max:1920, step:64}
#@@markdown CFG
CFG = 7  # @param {type:"slider", min:0, max:25, step:1}
#@markdown ***スケジューラ(サンプラー)***
scheduler="dpm++_2m_karras" #@param ["unipc","euler_a","euler","ddim","ddpm","deis","dpm2","dpm++_2s","dpm++_2m","dpm++_2m_karras","dpm++_sde","dpm++_sde_karras","heun","heun_karras","lms","lms_karras","pndm","dpm++_2m_karras"]
#@@markdown シード（-1の時はランダム）
seed = -1 #@param {type: "integer"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed
#@markdown ***プロンプトのトークン制限をどのくらい緩和するか → 75 × N になる***
max_embeddings_multiples=4  # @param {type:"slider", min:1, max:20, step:1}
#@markdown ***Hires.fix の有効化***
hires_fix = True #@param {type: "boolean"}
enhance_steps = 2 if hires_fix else 1
#@markdown ***解像度倍率(乗算後、最も近い8の倍数のサイズとなる)***
upscaling_ratio = 2 # @param {type:"slider", min:1, max:8, step:1}
if hires_fix == False:
  upscaling_ratio=1
#@markdown ***アップスケールステップ数***
up_steps = 20 # @param {type:"slider", min:5, max:100, step:5}
#@markdown ***小さいほど元画像を尊重） （0.5～0.7）が推奨***
denoising_strength = 0.6 # @param {type:"slider", min:0, max:2, step:0.1}

# 画像生成
for i in range(batch_count):
  if seed is None or seed == -1:valueSeed = inputSeed + i
  if isRandomPose and len(poses) > 0: poseIndex = random.randint(0, len(poses)-1)
  image = generator.diffusion_enhance(
    prompt,
    poses[poseIndex],
    negative,
    scheduler_name=scheduler,
    num_inference_steps=steps,
    num_inference_steps_enhance=up_steps,
    guidance_scale=CFG,
    width=img_width,
    height=img_height,
    seed=valueSeed,
    max_embeddings_multiples=max_embeddings_multiples,
    upscale_target="latent",
    interpolate_mode="bicubic",
    antialias=True,
    upscale_by=upscaling_ratio,
    enhance_steps=enhance_steps,
    denoising_strength=denoising_strength,
    output_type="pil",
    decode_factor=0.15,
    decode_factor_final=0.18215,
  )

  # 画像を保存

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)

  #@markdown ***メタデータの書き込み***
  save_metadata = True #@param {type: "boolean"}
  if save_metadata:
    metadata = PngInfo()
    # Model
    metadata.add_text("model",(model_name))
    # Lora
    if isUse_lora_1:
      metadata.add_text("lora_1",(lora_1))
      metadata.add_text("lora_alpha_1",(str(lora_alpha_1)))
    if isUse_lora_2:
      metadata.add_text("lora_2",(lora_2))
      metadata.add_text("lora_alpha_2",(str(lora_alpha_2)))
    if isUse_lora_3:
      metadata.add_text("lora_3",(lora_3))
      metadata.add_text("lora_alpha_3",(str(lora_alpha_3)))
    if isUse_lora_4:
      metadata.add_text("lora_4",(lora_4))
      metadata.add_text("lora_alpha_4",(str(lora_alpha_4)))
    if isUse_lora_5:
      metadata.add_text("lora_5",(lora_5))
      metadata.add_text("lora_alpha_5",(str(lora_alpha_5)))
    # embedding
    if isUse_embedding_1:
      metadata.add_text("embedding_1",(embedding_1))
      metadata.add_text("trigger_word_1",(str(trigger_word_1)))
    if isUse_embedding_2:
      metadata.add_text("embedding_2",(embedding_2))
      metadata.add_text("trigger_word_2",(str(trigger_word_2)))
    if isUse_embedding_3:
      metadata.add_text("embedding_3",(embedding_3))
      metadata.add_text("trigger_word_3",(str(trigger_word_3)))
    if isUse_embedding_4:
      metadata.add_text("embedding_4",(embedding_4))
      metadata.add_text("trigger_word_4",(str(trigger_word_4)))
    if isUse_embedding_5:
      metadata.add_text("embedding_5",(embedding_5))
      metadata.add_text("trigger_word_5",(str(trigger_word_5)))
    # プロンプト
    metadata.add_text("prompt",(prompt))
    metadata.add_text("negative",(negative))
    metadata.add_text("scheduler",(scheduler))
    metadata.add_text("steps",(str(steps)))
    metadata.add_text("CFG",(str(CFG)))
    metadata.add_text("width",(str(img_width)))
    metadata.add_text("height",(str(img_height)))
    metadata.add_text("seed",str((valueSeed)))
  if hires_fix:
    metadata.add_text("upscaling ratio",str((upscaling_ratio)))
    metadata.add_text("up steps",str((up_steps)))
    metadata.add_text("denoising strength",str((denoising_strength)))
    image.save(save_location, pnginfo=metadata)
  else:
    image.save(save_location)


In [ ]:
# @title 画像のメタデータを出力
# Driveに接続


import sys
import os
from PIL import Image
from PIL.PngImagePlugin import PngInfo
from google.colab import drive


isConectDrive=False  #@param {type: "boolean"}
if isConectDrive:
  drive.mount('/content/drive', force_remount=True)

#@markdown **保存した画像のパス**
file_dir = "/content/drive/MyDrive/StableDiffusion/txt2img_output"  #@param {type: "string"}
file_name = "20230726_195024_606196166.png" #@param {type: "string"}
file_path = os.path.join(file_dir, file_name)
if file_path:
  img = Image.open(file_path)

  # Model
  if "model" in img.text:
    print("model: ",img.text["model"])
  # LoRA
  if "lora_1" in img.text:
    print("lora_1: ",img.text["lora_1"])
    print("lora_alpha_1: ", img.text["lora_alpha_1"])
  if "lora_2" in img.text:
    print("lora_2: ",img.text["lora_2"])
    print("lora_alpha_2: ", img.text["lora_alpha_2"])
  if "lora_3" in img.text:
    print("lora_3: ",img.text["lora_3"])
    print("lora_alpha_3: ", img.text["lora_alpha_3"])
  if "lora_4" in img.text:
    print("lora_4: ",img.text["lora_4"])
    print("lora_alpha_4: ", img.text["lora_alpha_4"])
  if "lora_5" in img.text:
    print("lora_5: ",img.text["lora_5"])
    print("lora_alpha_5: ", img.text["lora_alpha_5"])
  # embedding
  if "embedding_1" in img.text:
    print("embedding_1: ",img.text["embedding_1"])
    print("trigger_word_1: ", img.text["trigger_word_1"])
  if "embedding_2" in img.text:
    print("embedding_2: ",img.text["embedding_2"])
    print("trigger_word_2: ", img.text["trigger_word_2"])
  if "embedding_3" in img.text:
    print("embedding_3: ",img.text["embedding_3"])
    print("trigger_word_3: ", img.text["trigger_word_3"])
  if "embedding_4" in img.text:
    print("embedding_4: ",img.text["embedding_4"])
    print("trigger_word_4: ", img.text["trigger_word_4"])
  if "embedding_5" in img.text:
    print("embedding_5: ",img.text["embedding_5"])
    print("trigger_word_5: ", img.text["trigger_word_5"])
  # Prompt
  print("Prompt: ",img.text["prompt"])
  print("Negative Prompt: ",img.text["negative"])
  if "scheduler" in img.text:
    print("Scheduler: ", img.text["scheduler"])
  print("Steps: ",img.text["steps"])
  print("CFG: ",img.text["CFG"])
  print("Width: ",img.text["width"])
  print("Height: ",img.text["height"])
  print("Seed: ",img.text["seed"])
  # Hires.fix
  try:
    print("Upscaling ratio; ",img.text["upscaling ratio"])
    print("Up steps: ",img.text["up steps"])
    print("Denoising strength: ",img.text["denoising strength"])
  except:
    print("Hires.fix was OFF.")

In [ ]:
# 画像処理用のライブラリをインポート
import cv2
import numpy as np
import matplotlib.pyplot as plt

# globモジュールをインポート
import glob

# natsortをインポートする
from natsort import natsorted

from google.colab import drive


isConectDrive=False  #@param {type: "boolean"}
if isConectDrive:
  drive.mount('/content/drive', force_remount=True)


# 画像ファイルのパスをリストに格納
# ここでは例として、フォルダ名がimagesで、PNG画像をすべて指定する
path = "/content/drive/MyDrive/StableDiffusion/txt2img_output/grid/*" #@param {type:"string"}
image_paths = []
for ext in ["png", "jpeg", "jpg"]:
  image_paths.extend(glob.glob(path + "." + ext))


if image_paths:
    # リストをファイル名で自然順にソートする
    image_paths = natsorted(image_paths)

    # 画像の高さの最小値と幅の最大値を初期化する
    min_height = float("inf")
    # min_height = 0 # 最小値を0に設定
    max_width = 0

    # image_pathsの画像の中で一番高さの低い画像の高さと一番幅の大きい画像の幅を取得する
    for path in image_paths:
        img = cv2.imread(path) # 画像を読み込む
        h, w, c = img.shape # 画像の高さ、幅、チャンネル数を取得する
        if h < min_height: # もし高さが最小値より小さい場合は
            min_height = h # 最小値を更新する
        if w > max_width: # もし幅が最大値より大きい場合は
            max_width = w # 最大値を更新する


    isAddBlack = False #@param {type: "boolean"}

    # 画像ファイルを読み込んで、リサイズしてリストに格納
    images = []
    for path in image_paths:
        img = cv2.imread(path) # 画像を読み込む
        h, w, c = img.shape # 画像の高さ、幅、チャンネル数を取得する
        scale = min_height / h # 縮小率を計算する
        new_h = min_height # 新しい高さは最小値とする
        new_w = int(w * scale) # 新しい幅を計算する（整数型に変換）
        img = cv2.resize(img, (new_w, new_h)) # 画像をリサイズする
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGRからRGBに色空間を変換する

        # # もし幅が最大値より小さい場合は、右側に黒い余白を追加する
        # if new_w < max_width:
        #     pad_width = max_width - new_w # 余白の幅を計算する（整数型に変換）
        #     pad_img = np.zeros((new_h, pad_width, 3), dtype=np.uint8) # 黒い余白の画像を作る
        #     img = np.hstack((img, pad_img)) # 元の画像と余白の画像を水平方向に結合する

        if isAddBlack:
          # もし幅が最大値より小さい場合は、左右に黒い余白を追加する
          if new_w < max_width:
              pad_width = max_width - new_w # 余白の幅を計算する（整数型に変換）
              pad_left = pad_width // 2 # 左側の余白の幅を計算する（整数型に変換）
              pad_right = pad_width - pad_left # 右側の余白の幅を計算する（整数型に変換）
              pad_img_left = np.zeros((new_h, pad_left, 3), dtype=np.uint8) # 左側の黒い余白の画像を作る
              pad_img_right = np.zeros((new_h, pad_right, 3), dtype=np.uint8) # 右側の黒い余白の画像を作る
              img = np.hstack((pad_img_left, img, pad_img_right)) # 左側の余白、元の画像、右側の余白を水平方向に結合する

        images.append(img) # リストに追加する


    # 画像をグリッド表示するためのパラメータを設定
    # 列数
    rows = 2 #@param {type:"integer"}
    # 行数
    cols = 2 #@param {type:"integer"}
    required_images = 4 #@param {type:"integer"}


    # 画像が足りない場合は、黒い画像を作ってリストに追加する
    num_images = len(images) # リストに格納された画像の数
    if num_images < required_images: # もし足りない場合は
        for i in range(required_images - num_images): # 足りない分だけ繰り返す（整数型に変換）
            black_img = np.zeros((int(min_height), int(max_width), 3), dtype=np.uint8) # 黒い画像を作る（整数型に変換）
            images.append(black_img) # リストに追加する

    # 画像をグリッド表示するための関数を定義
    def grid_display(images, rows, cols):
        # 画像の数と行列数が一致しない場合はエラーを出す
        assert len(images) == rows*cols

        # 行列数に応じて画像を結合する
        # まずは行ごとに画像を水平方向に結合する
        horizontal = []
        for i in range(rows):
            start_index = i*cols # 行の先頭の画像のインデックス
            end_index = start_index + cols # 行の最後の画像のインデックス
            horizontal.append(np.hstack(images[start_index:end_index])) # 水平方向に結合した画像をリストに追加する

        # 次に、結合した行ごとの画像を垂直方向に結合する
        vertical = np.vstack(horizontal) # 垂直方向に結合した画像

        # 結果を返す
        return vertical

    # 関数を呼び出して、画像をグリッド表示する
    result = grid_display(images, cols, rows) # 関数を呼び出す
    plt.figure(figsize=(50,50)) # 図のサイズを設定する
    plt.imshow(result) # 図に画像を表示する（補間なし）
    plt.axis("off") # 軸を非表示にする
    plt.show() # 図を表示する


In [ ]:
#@title Restart session
#@markdown Executing this cell crashes session. It's intended.<br>You don't need to re-execute cells above.
import os
os._exit(00)